# So Your Email Has Been Stolen...

We hear a lot about data breaches, about a company exposing our email addresses or IP addresses. We heave a sigh of relief that it was _"just"_ email addresses or IP addresses. Our passwords and financial data wasn't in danger. This project starts off with a very basic form of _OSINT_*: what kind of information can attackers find just starting off with the humble email address?

I started off with Pastebin, a text-sharing site where a lot of these stolen information gets dumped, and found two recent breaches that exposed email addresses and IP addresses.

*\*Open Source Intelligence*


In [8]:
#import all here

import numpy as np
import pandas as pd

import re

import requests

# no longer needed (see Path Not Taken, 1)
#from ipaddress import IPv4Address

from dotenv import load_dotenv
import os

In [10]:
# get my API Key from the .env 
load_dotenv()

IP_KEY = os.getenv("IPSTACK_API_KEY")

#check that I read it correctly. Rerun it later to remove this from the output.
#print(IP_KEY)

## Preparing the data files

I could've scraped the site, but Pastebin provides the dumps as text files. I used multiple search-and-replace regex strings in _vi_ to fix the layout of the first file to turn it into a proper CSV file. I used a different set of regex search-and-replace to fix the second file.

#### Old Layout for file 1:
```
userid
password
email address
ip address
----------- 
userid
```

#### New file layout:
```
"username","password","email","ip"
```

In [5]:
#read in first file
df1 = pd.read_csv("data/cleaned_data.csv")
df1.head()

,username,password,email,ip
0,alan,alanpw,alann@,186.18.8.95
1,lucasluquitas97,40809023lucas,abadisagustin@gmail.com,181.168.120.106
2,lucasluquitas97,40809023lucas,abadisagustin@gmail.com,181.168.120.106
3,killerselta03,agustin123,abadisagustin@gmail.com,181.168.120.106
4,lucasluquitas97,40809023lucas,agustin.n.abadia,181.168.120.106


#### Old layout for file 2:
```
+-------------------------------------------+-----------------+
| email                                     | ip              |
+-------------------------------------------+-----------------+
| email                                     | ip.address      |
```

#### New file layout:
```
"email","ip"
```

The biggest challenge with this clean-up process was to find misspellings of email addresses, such as _cm_ instead of _com.

In [6]:
#read in second file
df2 = pd.read_csv("data/cleaned_edu_dump.csv")
df2.head()

,email,ip
0,kevincho@scripps.edu,137.131.20.189
1,metlin@scripps.edu,137.131.20.189
2,Alfons.Hester@anorg.chemie.uni-giessen.de,134.176.172.237
3,bluelou@gmx.de,130.60.81.126
4,mfernandez.eps@ceu.es,193.146.228.26


The first file needed the most cleanup. We removed the username and password -- for all intents in purposes, I didn't want to be trafficking in user passwords. I also performed a deduplicate function, which would fix the rows dumping into the same email.

In [7]:
# prepare first file to look like the second by dropping extra columns
df1 = df1.drop(["username","password"], axis=1)

In [8]:
#vertical joining the dataframes to create one master dataframe
df = df1.append(df2)

In [9]:
#there are fields with null email. Drop them
df = df.dropna(subset=['email'])

#during the cleanup of the files, I didn't notice there are invalid email addresses. Remove those rows
df = df[df.email.str.contains("\w*[@]\w*[.]\w*")]

# there are also duplicates and null values. Clean up the files.
df.sort_values("email")
df = df.drop_duplicates(subset="email")

df.head(10)

,email,ip
1,abadisagustin@gmail.com,181.168.120.106
5,Carloselmandril@gmail.com,186.48.125.117
6,cacacasdfa@hotmail.com,186.130.6.73
8,ignaciomortega95@gmail.com,186.18.108.185
9,yerkhomolina1990@gmail.com,181.74.174.95
11,sebas12341992@hotmail.com,181.49.73.230
13,troll@hotmail.com,190.18.78.110
14,j.angulo.valdivia@gmail.com,190.209.55.16
19,akantoresg@gmail.com,179.59.169.156
23,thiagojosso@hotmail.com,190.138.92.72


## Mapping the IP Address to a Geolocation

### Path Not Taken \[1\]

Originally, I obtained a Geolocation database file maintained by MaxMind at [https://dev.maxmind.com/geoip/geoip2/geolite2/](https://dev.maxmind.com/geoip/geoip2/geolite2/). The file contains IP address ranges. Since it is easier to working with IP addresses when using the integer format, instead of quad format, I ran the conversion utility on the Geolocation database to convert the IP address ranges into integer format. [https://github.com/maxmind/geoip2-csv-converter](https://github.com/maxmind/geoip2-csv-converter)

```
geodata_master = pd.read_csv("data/GeoLite-Blocks-IPv4.csv")
```

I also converted the master dataframe to put all the IP addresses into integer format using the following:

```
df['ip_int'] = df.apply(lambda row: int(IPv4Address(row.ip)), axis = 1)
```

I tried using the _df_ dataframe and the _geodata_master_ dataframe and the comparison was getting tricky.

```
df['latitude'].apply(lambda latitude: geodata_master['latitude'][(geodata_master['network_start_integer'] <= find_int) & (geodata_master['network_last_integer'] >= find_int)].values[0])
```

I switched the master dataframe to a list to make it easier to work with.

```
rows = df.to_dict('records')
```

I then looped through all the rows to look up the value in the geodata database and added the longitude and latitude fields.

```
    find_int = row['ip_int']
    found = geodata.loc[(geodata['network_start_integer'] < find_int) & (geodata['network_last_integer'] > find_int)]
    row.update({'longitude':found['longitude'].values[0]})
    row.update({'latitude':found['latitude'].values[0]})

df = pd.DataFrame(rows)
```

This worked perfectly well, **except** the database isn't very comprehensive and some IP addresses couldn't find a corresponding location information. 

### Using the API

That is when I switched to using an API from IPStack to handle the lookup. This one is set up to use the regular quad-format IP address to obtain the longitude and latitude. The API is set up to give the flag, state, city, and other demographic information such as the language spoken.

In [ ]:
#switch to list because it is easier to work a list than a dataframe
rows = df.to_dict('records')

In [10]:
#logic to get the IP location into this dataframe, and written to a file

for row in rows:
    find_ip = row['ip']
    response = requests.get(f'http://api.ipstack.com/{find_ip}?access_key={IP_KEY}')
    place = response.json()
    row.update({'longitude':place['longitude']})
    row.update({'latitude':place['latitude']})
    
df = pd.DataFrame(rows)
df

,email,ip,latitude,longitude
0,abadisagustin@gmail.com,181.168.120.106,-38.949261,-68.059479
1,Carloselmandril@gmail.com,186.48.125.117,-34.883999,-56.162998
2,cacacasdfa@hotmail.com,186.130.6.73,-34.504719,-58.679520
3,ignaciomortega95@gmail.com,186.18.108.185,-34.687401,-58.563301
4,yerkhomolina1990@gmail.com,181.74.174.95,-33.465000,-70.655998
5,sebas12341992@hotmail.com,181.49.73.230,4.600000,-74.083328
6,troll@hotmail.com,190.18.78.110,-34.661098,-58.367001
7,j.angulo.valdivia@gmail.com,190.209.55.16,-33.465000,-70.655998
8,akantoresg@gmail.com,179.59.169.156,-16.500000,-68.150002
9,thiagojosso@hotmail.com,190.138.92.72,-31.405251,-64.180389


## Writing the Data File

The master dataframe creates the main table in my database schema

In [11]:
# write this to new csv
df.to_csv('maintable.csv', index=0)